<a href="https://colab.research.google.com/github/MCygert/handball_classifier/blob/main/handball_video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project made for classifing actions in handball


## Loading data

In [4]:
!ls

drive  sample_data


In [5]:
directory_to_data = "drive/MyDrive/handball"
!ls drive/MyDrive/handball

passes	saves  shots


## 1. We need to split data 80/20 into folders


In [6]:
import os
import shutil

# Create folders for test and train data
os.mkdir('test')
os.mkdir('train')

# Creating subfolders per class in both folders
directories = []
for dirname, dirpath, filename in os.walk(directory_to_data):
  directories.append(dirpath)
directories = directories[0]

for directory in directories:
  os.mkdir(f'test/{directory}')
  os.mkdir(f'train/{directory}')

In [7]:
files = []
sorted_by_classes = {}
for dirname, dirpath, filename in os.walk(directory_to_data):
  files.append(filename)
files = files[1:]
# Create dictionary which has{class: list_of_videos}
for i, dictionary in enumerate(directories):
  sorted_by_classes[dictionary] = files[i]  


In [8]:
import random
def split_data(data):
  '''
  Returning 2 lists which are splitted in 10/90 manner 
  '''
  train_data = data.copy()
  test_data = []
  for i in range(round(len(data) / 10)):
    test_data.append(train_data.pop(random.randrange(len(train_data))))
  return train_data, test_data

In [9]:
train_data, test_data = split_data(sorted_by_classes['saves'])

# Check if it's splitting correctly
len(train_data), len(test_data), len(sorted_by_classes['saves'])

(49, 5, 54)

In [10]:
# Create splitted data
train_elements = {}
test_elements = {}
for key in sorted_by_classes.items():
  class_of_video = key[0]
  train, test = split_data(key[1])
  train_elements[class_of_video] = train
  test_elements[class_of_video] = test
test_elements['saves'], test_elements['passes'], test_elements['shots']

(['save54.mp4', 'save29.mp4', 'save47.mp4', 'save43.mp4', 'save11.mp4'],
 ['pass71.mp4',
  'pass152.mp4',
  'pass303.mp4',
  'pass10.mp4',
  'pass122.mp4',
  'pass274.mp4',
  'pass283.mp4',
  'pass160.mp4',
  'pass215.mp4',
  'pass149.mp4',
  'pass302.mp4',
  'pass191.mp4',
  'pass61.mp4',
  'pass81.mp4',
  'pass69.mp4',
  'pass154.mp4',
  'pass228.mp4',
  'pass6.mp4',
  'pass236.mp4',
  'pass73.mp4',
  'pass342.mp4',
  'pass50.mp4',
  'pass36.mp4',
  'pass107.mp4',
  'pass138.mp4',
  'pass207.mp4',
  'pass33.mp4',
  'pass199.mp4',
  'pass292.mp4',
  'pass328.mp4',
  'pass264.mp4',
  'pass3.mp4',
  'pass48.mp4',
  'pass234.mp4',
  'pass84.mp4'],
 ['shot69.mp4',
  'shot115.mp4',
  'shot52.mp4',
  'shot18.mp4',
  'shot114.mp4',
  'shot39.mp4',
  'shots13.mp4',
  'shot46.mp4',
  'shot35.mp4',
  'shot63.mp4',
  'shot77.mp4',
  'shot97.mp4',
  'shot37.mp4',
  'shot6.mp4',
  'shot12 - SD 480p.mov'])

In [11]:
def move_files_to_correct_directories(data_dictionary, type_of_data):
  for key in data_dictionary.items():
    for file_name in key[1]:
      shutil.copy(directory_to_data + '/' + key[0] + '/' + file_name, type_of_data + '/' + key[0] + '/' + file_name)

In [12]:
# Moving files into correct folders
move_files_to_correct_directories(train_elements, 'train')
move_files_to_correct_directories(test_elements, 'test')

## 2. Extract jpegs of videos
Later I can extract frames and save them as jpegs from video and move them to folder. After that get features with CNN it could save a lot of time while processing the data. Because it will go straight to RNN layers 


## 3. Create csv 
Which will have  **label,path** structure per train and test directory


In [13]:
test_files = {}
for dirname, dirpath, filename in os.walk("test"):
  print(dirname)
  test_files[dirname] = filename
test_files.pop('test')
for key in test_files:
  print(key)
  test_files[key.split('/')[1]] = test_files.pop(key)

print(test_files.keys()) 

test
test/shots
test/saves
test/passes
test/shots
test/saves
dict_keys(['test/passes', 'shots', 'saves'])


In [14]:
def get_files_path(path):
  list_of_paths = []
  for root, dirs, files in os.walk("train/" + path):
    for filename in files:
      list_of_paths.append(root + '/' + filename)
  return list_of_paths

In [20]:
# Trying to create correct label
import pandas as pd
map_for_paths = {"shots": get_files_path('shots'),
                 'saves': get_files_path('saves'),
                 'passes': get_files_path('passes')}

list_of_tuples_for_data_frame = []
shots = map_for_paths['shots']
saves = map_for_paths['saves']
passes = map_for_paths['passes']
for path in shots:
  list_of_tuples_for_data_frame.append(('shots', path))
for path in saves:
  list_of_tuples_for_data_frame.append(('saves', path))
for path in passes:
  list_of_tuples_for_data_frame.append(('passes', path))
list_of_tuples_for_data_frame[:5]
df = pd.DataFrame(list_of_tuples_for_data_frame, columns=['Tag', 'Path'])  
df.shape

(500, 2)

# Create models for training

##1. Create data loader

##2. Create feature extractor with pre-trained model

##3. Feed extracted features to rnn model